
# Data Processor For NSUN6 Model Training Data


[EXPLAIN]

The following are adjustable values (best set to 50, 0.6)... 

In [ ]:
sample_count_threshold = 50;
NSUN6_NSUN2_cutoff_factor = 0.6;
pre_processed_data = "ubsseq_preprocessed_data.xlsx"


### Imports & Functions


In [ ]:
import random
import math
import pandas as pd
import numpy as np
from pyfaidx import Fasta
import matplotlib.pyplot as plt
from matplotlib import colors
import logomaker as lm
import re

In [ ]:
def count_iterable(i):
    return sum(1 for e in i)

In [ ]:
PATH = '../GenomeData/'
def extract_seq_genome(genome, chrom, start, end, strand = '+'):
    '''
    Usage:
        - If strand == '+', return sequence
        - If strand == '-', return sequence revese complementary
        - With hg38.get_seq(chrom, start, end, rc= strand == '-').seq get correct position of miRNA,
        for mapping result, you should modify by yourself.

    @author: Trung Duc Nguyen
    '''

    if genome == 'hsa':
        genome = 'hg38'

    gene = Fasta(f"{PATH}/{genome}.fa") 
    #hg38.get_seq('chr1', 100009588 + 1, 100009608, rc=False).seq
    return(gene.get_seq(chrom, start+1, end, rc= strand == '-').seq)


## Data Pre-Processing



### Remove Less-Sampled Genes


In [ ]:
data_raw = pd.read_excel(pre_processed_data)
data_raw_shape = data_raw.shape

In [ ]:
# Remove data sampled less than threshold
data_sample_count = data_raw[['unconverted', 'converted']].sum(axis=1)
data_significant = data_raw[data_sample_count > sample_count_threshold]
# Remove data with unspecified chromosome
data_significant = data_significant[data_significant['chrom'] != 'MT']
data_significant_shape = data_significant.shape
print(f"Filtered data contains {data_significant_shape[0]} genes out of an initial {data_raw_shape[0]} genes, a {data_significant_shape[0]/data_raw_shape[0]*100:.2f}% retention")


### Sort Likely NSUN6 Affected Genes By Delta Ratio



## Sequence Extraction



## Structure Prediction



## Final Formatting
